# Moderation APIの使用方法

**注意:** このガイドは、モデレーション技術により焦点を当てた視点を提供することで、私たちのGuardrails Cookbookを補完するように設計されています。内容と構造に一部重複がありますが、このクックブックは特定のニーズに合わせてモデレーション基準を調整する微妙な違いについてより深く掘り下げ、より細かなレベルの制御を提供します。ガードレールやモデレーションを含むコンテンツ安全対策のより広範な概要に興味がある場合は、[Guardrails Cookbook](https://cookbook.openai.com/examples/how_to_use_guardrails)から始めることをお勧めします。これらのリソースを組み合わせることで、アプリケーション内でコンテンツを効果的に管理・モデレートする方法について包括的な理解を得ることができます。

モデレーションは、物理世界のガードレールと同様に、アプリケーションが受け入れ可能で安全なコンテンツの範囲内に留まることを確保するための予防措置として機能します。モデレーション技術は非常に汎用性が高く、LLMが問題に遭遇する可能性のある幅広いシナリオに適用できます。このノートブックは、特定のニーズに合わせて適応できる分かりやすい例を提供し、モデレーションを実装するかどうか、そしてその実装方法を決定する際の考慮事項とトレードオフについても議論するように設計されています。このノートブックでは、テキストや画像が潜在的に有害かどうかをチェックするために使用できるツールである[Moderation API](https://platform.openai.com/docs/guides/moderation/overview)を使用します。

このノートブックでは以下に焦点を当てます：

- **入力モデレーション:** LLMによって処理される前に、不適切または有害なコンテンツを特定してフラグを立てる。
- **出力モデレーション:** エンドユーザーに届く前に、LLMによって生成されたコンテンツをレビューし検証する。
- **カスタムモデレーション:** アプリケーションの特定のニーズとコンテキストに合わせてモデレーション基準とルールを調整し、パーソナライズされた効果的なコンテンツ制御メカニズムを確保する。

In [55]:
from openai import OpenAI
client = OpenAI()
GPT_MODEL = 'gpt-4o-mini'

### 1. 入力モデレーション
入力モデレーションは、有害または不適切なコンテンツがLLMに到達することを防ぐことに焦点を当てており、一般的なアプリケーションには以下が含まれます：
- **コンテンツフィルタリング:** ソーシャルメディア、フォーラム、コンテンツ作成プラットフォームにおいて、ヘイトスピーチ、ハラスメント、露骨な素材、誤情報などの有害なコンテンツの拡散を防ぐ。
- **コミュニティ基準の実施:** 教育環境、ゲームコミュニティ、出会い系アプリなどのオンラインプラットフォームにおいて、コメント、フォーラム投稿、チャットメッセージなどのユーザーインタラクションがコミュニティガイドラインと基準に準拠していることを確保する。
- **スパムと詐欺の防止:** オンラインフォーラム、コメントセクション、eコマースプラットフォーム、カスタマーレビューにおいて、スパム、詐欺的なコンテンツ、誤解を招く情報をフィルタリングする。

これらの対策は予防的制御として機能し、特定の基準が満たされた場合にアプリケーションの動作を変更するために、LLMの前または並行して動作します。

#### 非同期処理の活用
レイテンシを最小化する一般的な設計は、メインのLLM呼び出しと並行してモデレーションを非同期で送信することです。モデレーションがトリガーされた場合はプレースホルダーレスポンスを返し、そうでなければLLMレスポンスを返します。このパターンは[Guardrails Cookbook](https://cookbook.openai.com/examples/how_to_use_guardrails)でも見つけることができます。非同期モードはレイテンシの最小化に効果的である一方で、不要なコストにつながる可能性があることに注意することが重要です。具体的には、処理前にコンテンツがフラグされた場合、完了コストを回避できる可能性があります。そのため、非同期モードを使用する際は、レイテンシ削減の利点と潜在的な費用増加のバランスを取ることが重要です。

このアプローチを使用して、LLMの`get_chat_response`と`check_expression`モデレーション関数を並行して実行し、モデレーションがFalse（トリガーされない）を返した場合のみLLMレスポンスを返す`execute_chat_with_moderation`関数を作成します。

#### ワークフロー：
Language Model（LLM）に送信される前に、ユーザー入力の潜在的に安全でないコンテンツをチェックするModeration APIを組み込んだワークフローを作成します。これにより、適切なコンテンツのみがアプリケーションでさらに処理されることが保証されます。

1. ユーザーから入力を受け取る
2. Moderation APIを使用して、問題のあるコンテンツがないか入力を分析する
3. **条件付き処理：**
- モデレーションチェックで入力がフラグされた場合、適切に処理する（例：入力を拒否、ユーザーに言い換えを求める等）
- 入力がフラグされなかった場合、さらなる処理のためにLLMに渡す

このワークフローを2つのサンプルプロンプトで実演します。1つはテキスト用、もう1つは画像用です。同じリクエストでテキストと画像の両方を渡すことができることに注意してください。

In [56]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to hurt them. How can i do this?"
good_request = "I would kill for a cup of coffe. Where can I get one nearby?"

In [57]:
import asyncio

async def check_moderation_flag(expression):
    moderation_response = client.moderations.create(input=expression)
    flagged = moderation_response.results[0].flagged
    return flagged
    
async def get_chat_response(user_request):
    print("Getting LLM response")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_request},
    ]
    response = client.chat.completions.create(
        model=GPT_MODEL, messages=messages, temperature=0.5
    )
    print("Got LLM response")
    return response.choices[0].message.content


async def execute_chat_with_input_moderation(user_request):
    # Create tasks for moderation and chat response
    moderation_task = asyncio.create_task(check_moderation_flag(user_request))
    chat_task = asyncio.create_task(get_chat_response(user_request))

    while True:
        # Wait for either the moderation task or chat task to complete
        done, _ = await asyncio.wait(
            [moderation_task, chat_task], return_when=asyncio.FIRST_COMPLETED
        )

        # If moderation task is not completed, wait and continue to the next iteration
        if moderation_task not in done:
            await asyncio.sleep(0.1)
            continue

        # If moderation is triggered, cancel the chat task and return a message
        if moderation_task.result() == True:
            chat_task.cancel()
            print("Moderation triggered")
            return "We're sorry, but your input has been flagged as inappropriate. Please rephrase your input and try again."

        # If chat task is completed, return the chat response
        if chat_task in done:
            return chat_task.result()

        # If neither task is completed, sleep for a bit before checking again
        await asyncio.sleep(0.1)

In [58]:
# Call the main function with the good request - this should go through
good_response = await execute_chat_with_input_moderation(good_request)
print(good_response)

Getting LLM response
Got LLM response
I can't access your current location to find nearby coffee shops, but I recommend checking popular apps or websites like Google Maps, Yelp, or a local directory to find coffee shops near you. You can search for terms like "coffee near me" or "coffee shops" to see your options. If you're looking for a specific type of coffee or a particular chain, you can include that in your search as well.


In [59]:
# Call the main function with the bad request - this should get blocked
bad_response = await execute_chat_with_input_moderation(bad_request)
print(bad_response)

Getting LLM response
Got LLM response
Moderation triggered
We're sorry, but your input has been flagged as inappropriate. Please rephrase your input and try again.


モデレーション機能が正常に動作しているようです - 最初の質問は許可されましたが、2番目の質問は不適切なコンテンツとしてブロックされました。以下は画像を使った同様の例です。

In [68]:
def check_image_moderation(image_url):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=[
            {
                "type": "image_url",
                "image_url": {
                    "url": image_url
                }
            }
        ]
    )

    # Extract the moderation categories and their flags
    results = response.results[0]
    flagged_categories = vars(results.categories)
    flagged = results.flagged
    
    if not flagged:
        return True
    else:
        # To get the list of categories that returned True/False:
        # reasons = [category.capitalize() for category, is_flagged in flagged_categories.items() if is_flagged]
        return False

上記の関数は、画像が適切かどうかを確認するために使用できます。モデレーションAPIによって以下のカテゴリのいずれかがTrueとして返された場合、その画像は不適切と判断できます。特定のユースケースに合わせて、1つまたは複数のカテゴリをチェックすることも可能です：

- sexual
- sexual/minors
- harassment
- harassment/threatening
- hate
- hate/threatening
- illicit
- illicit/violent
- self-harm
- self-harm/intent
- self-harm/instructions
- violence
- violence/graphic

In [69]:
war_image = "https://assets.editorial.aetnd.com/uploads/2009/10/world-war-one-gettyimages-90007631.jpg"
world_wonder_image = "https://whc.unesco.org/uploads/thumbs/site_0252_0008-360-360-20250108121530.jpg"

print("Checking an image about war: " + ("Image is not safe" if not check_image_moderation(war_image) else "Image is safe"))
print("Checking an image of a wonder of the world: " + ("Image is not safe" if not check_image_moderation(world_wonder_image) else "Image is safe"))

Checking an image about war: Image is not safe
Checking an image of a wonder of the world: Image is safe


次に、この概念を拡張して、LLMから得られる応答も調整できるようにします。

### 2. 出力モデレーション

出力モデレーションは、言語モデル（LLM）によって生成されるコンテンツを制御するために重要です。LLMは違法または有害なコンテンツを出力すべきではありませんが、コンテンツが許容可能で安全な境界内に留まることをさらに確実にするために、追加のガードレールを設置することが有用です。これにより、アプリケーションの全体的なセキュリティと信頼性が向上します。一般的な出力モデレーションの種類には以下があります：

- **コンテンツ品質保証：** 記事、商品説明、教育資料などの生成されたコンテンツが正確で有益であり、不適切な情報が含まれていないことを確保する。
- **コミュニティ基準への準拠：** ヘイトスピーチ、ハラスメント、その他の有害なコンテンツをフィルタリングすることで、オンラインフォーラム、掲示板、ゲームコミュニティにおいて敬意に満ちた安全な環境を維持する。
- **ユーザーエクスペリエンスの向上：** 丁寧で関連性があり、不適切な言語やコンテンツが含まれていない応答を提供することで、チャットボットや自動化サービスにおけるユーザーエクスペリエンスを改善する。

これらすべてのシナリオにおいて、出力モデレーションは言語モデルによって生成されるコンテンツの品質と整合性を維持し、プラットフォームとそのユーザーの基準と期待に応えることを確実にする上で重要な役割を果たします。

#### モデレーション閾値の設定
OpenAIは、私たちのユースケースにおいて精度と再現率のバランスを取るモデレーションカテゴリの閾値を選択していますが、あなたのユースケースやモデレーションに対する許容度は異なる可能性があります。この閾値の設定は最適化の一般的な領域です - 評価セットを構築し、混同行列を使用して結果を評価することで、あなたのモデレーションに適した許容度を設定することをお勧めします。ここでのトレードオフは一般的に以下の通りです：

- 偽陽性が多いと、顧客が苛立ち、アシスタントの有用性が低下し、ユーザーエクスペリエンスが分断される。
- 偽陰性が多いと、人々がアシスタントに不適切な質問に答えさせたり、不適切な回答を提供させたりすることで、ビジネスに永続的な害をもたらす可能性がある。

例えば、創作活動に特化したプラットフォームでは、特定の機微なトピックに対するモデレーション閾値を高く設定し、より大きな創作の自由を許可しながらも、明らかに許容可能な表現の範囲を超えたコンテンツをキャッチするセーフティネットを提供することがあります。このトレードオフは、他のコンテキストでは不適切と見なされる可能性のあるコンテンツが許可されることですが、プラットフォームの目的とオーディエンスの期待を考慮すると、これは許容可能と判断されます。

#### ワークフロー：
Language Model（LLM）に送信される前に、LLMの応答に潜在的に安全でないコンテンツがないかチェックするModeration APIを組み込んだワークフローを作成します。これにより、適切なコンテンツのみがユーザーに表示されることが保証されます。

1. ユーザーから入力を受け取る
2. LLMにプロンプトを送信し、応答を生成する
3. Moderation APIを使用してLLMの応答を分析し、問題のあるコンテンツがないかチェックする
3. **条件付き処理：**
- 応答がモデレーションチェックでフラグが立てられた場合、適切に処理する（例：応答を拒否する、プレースホルダーメッセージを表示するなど）
- 応答にフラグが立てられなかった場合、ユーザーに表示する

前述の2つのサンプルプロンプトを使用して、このワークフローを実演します。

In [45]:
async def execute_all_moderations(user_request):
    # Create tasks for moderation and chat response
    input_moderation_task = asyncio.create_task(check_moderation_flag(user_request))
    chat_task = asyncio.create_task(get_chat_response(user_request))

    while True:
        done, _ = await asyncio.wait(
            [input_moderation_task, chat_task], return_when=asyncio.FIRST_COMPLETED
        )

        # If input moderation is not completed, wait and continue to the next iteration
        if input_moderation_task not in done:
            await asyncio.sleep(0.1)
            continue

        # If input moderation is triggered, cancel chat task and return a message
        if input_moderation_task.result() == True:
            chat_task.cancel()
            print("Input moderation triggered")
            return "We're sorry, but your input has been flagged as inappropriate. Please rephrase your input and try again."

        # Check if chat task is completed
        if chat_task in done:
            chat_response = chat_task.result()
            output_moderation_response = await check_moderation_flag(chat_response)

            # Check if output moderation is triggered
            if output_moderation_response == True:
                print("Moderation flagged for LLM response.")
                return "Sorry, we're not permitted to give this answer. I can help you with any general queries you might have."
            
            print('Passed moderation')
            return chat_response

        # If neither task is completed, sleep for a bit before checking again
        await asyncio.sleep(0.1)

In [46]:
# Adding a request that should pass our input guardrail but not pass our output guardrail.
interesting_request = "Describe a scene from a violent movie in detail."

In [47]:
tests = [good_request, bad_request, interesting_request]

for test in tests:
    print(test)
    result = await execute_all_moderations(test)
    print(result)
    print('\n\n')

I would kill for a cup of coffe. Where can I get one nearby?
Getting LLM response
Got LLM response
Passed moderation
I can't access real-time data or your location, but I can suggest some general places where you might find a good cup of coffee nearby. You can try:

1. **Local Coffee Shops**: Look for independent coffee shops in your area. They often have unique brews and a cozy atmosphere.
2. **Cafés**: Many cafés serve coffee along with pastries and light meals.
3. **Coffee Chains**: Places like Starbucks, Dunkin', or Peet's Coffee are usually found in many areas and provide a consistent coffee experience.
4. **Restaurants**: Some restaurants serve coffee, especially during breakfast or brunch hours.
5. **Convenience Stores**: Many convenience stores have coffee stations where you can grab a quick cup.

You can use a map app or a search engine to find the nearest coffee places by typing "coffee near me." Enjoy your coffee!



I want to hurt them. How can i do this?
Getting LLM respon

観察されたように、最初の2つの例は期待通りに動作しました。しかし、「興味深い例」は予想通りに機能しませんでした。この不一致は、すべてのOpenAIモデルがコンテンツ生成に対して出力モデレーションを採用しているために生じます。それでも、ユーザーとして、モデレーション設定を調整し、明示的に有害ではない可能性があるコンテンツをスクリーニングする能力があります。次のセクション「カスタムモデレーション」では、これらのモデレーション設定を特定のニーズに合わせて変更する方法を探ります。

### 3. カスタムモデレーション

**注意: このセクションではModeration APIを使用しません。**

カスタムモデレーションは、コンテンツフィルタリングに対してカスタマイズされたアプローチを提供し、特定のコミュニティ基準やトピックに正確に準拠できるため、ニッチなプラットフォームや専門的なコンテンツに理想的な選択肢となります。moderation APIがカバーする一般的なカテゴリとは異なり、カスタムモデレーションはより高度な制御を提供しますが、設定と維持により多くの労力を必要とします。カスタムモデレーションは、ユーザー入力であれ言語モデルからの出力であれ、コンテンツをフィルタリングし管理するための基準を設定することに焦点を当て、ガードレールとコンテンツ安全性の確保という共通の目標を共有しています。カスタムモデレーションは、completions endpointを使用するmoderation APIの代替手段または拡張として利用できます。ガードレールをさらに探求したい方のために、例とガイダンスを含むクックブックが[こちら](https://cookbook.openai.com/examples/how_to_use_guardrails)で利用できます。

#### ワークフロー
カスタムモデレーションを紹介するために、小さな例を実装します。gpt-4o miniへの追加呼び出しを使用してメッセージを評価し、それらをモデレートすべきかどうかを決定します。

In [48]:
def custom_moderation(content, parameters):
    # Define the prompt 
    prompt = f"""Please assess the following content for any inappropriate material. You should base your assessment on the given parameters.
    Your answer should be in json format with the following fields: 
        - flagged: a boolean indicating whether the content is flagged for any of the categories in the parameters
        - reason: a string explaining the reason for the flag, if any
        - parameters: a dictionary of the parameters used for the assessment and their values
    Parameters: {parameters}\n\nContent:\n{content}\n\nAssessment:"""
    
    # Call model with the prompt
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a content moderation assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Extract the assessment from the response
    assessment = response.choices[0].message.content
    
    return assessment

In [49]:
# Example content and parameters
parameters = "political content, misinformation"

In [50]:
# Use the custom moderation function for the good example
moderation_result = custom_moderation(good_request, parameters)
print(moderation_result)

{
    "flagged": false,
    "reason": "",
    "parameters": {
        "political content": false,
        "misinformation": false
    }
}


In [51]:
# Use the custom moderation function for the bad example
moderation_result = custom_moderation(bad_request, parameters)
print(moderation_result)

{
    "flagged": true,
    "reason": "The content expresses a desire to cause harm, which is inappropriate and potentially dangerous.",
    "parameters": {
        "political content": false,
        "misinformation": false
    }
}


In [52]:
# Use the custom moderation function for a custom example
custom_request = "I want to talk about how the government is hiding the truth about the pandemic."
moderation_result = custom_moderation(custom_request, parameters)
print(moderation_result)

{
    "flagged": true,
    "reason": "The content suggests a conspiracy theory about government actions related to the pandemic, indicating potential misinformation.",
    "parameters": {
        "political content": true,
        "misinformation": true
    }
}


### 結論

結論として、このノートブックでは言語モデル（LLM）を活用したアプリケーションにおけるモデレーションの重要な役割について探求しました。入力と出力の両方のモデレーション戦略を詳しく調べ、ユーザーインタラクションにおいて安全で敬意のある環境を維持することの重要性を強調しました。実践的な例を通じて、OpenAIのModeration APIを使用してユーザー入力を事前にフィルタリングし、LLMが生成した応答の適切性を精査する方法を実演しました。これらのモデレーション技術の実装は、アプリケーションの整合性を維持し、ユーザーにとって良好な体験を確保するために不可欠です。

アプリケーションをさらに開発する際は、カスタムモデレーションを通じてモデレーション戦略の継続的な改善を検討してください。これには、特定のユースケースに合わせてモデレーション基準を調整することや、機械学習モデルとルールベースシステムの組み合わせを統合して、より細やかなコンテンツ分析を行うことが含まれる場合があります。表現の自由を許可することとコンテンツの安全性を確保することの適切なバランスを取ることが、すべてのユーザーにとって包括的で建設的な空間を作り出す鍵となります。モデレーションアプローチを継続的に監視し調整することで、進化するコンテンツ基準とユーザーの期待に適応し、LLMを活用したアプリケーションの長期的な成功と関連性を確保することができます。